In [ ]:
import pickle
from pandas import DataFrame
import pandas


pandas.options.mode.copy_on_write = True
pandas.set_option("future.no_silent_downcasting", True)
try:
    with open("data/bloomData.pkl", "rb") as dataPickle:
        fullData: DataFrame = pickle.load(dataPickle, fix_imports=True)
except:
    fullData = DataFrame()

## Filter to important entries

In [ ]:
fullData = fullData.replace({"EconomyBan": {"banned": True, "none": False}}).infer_objects(copy=False)



data = fullData.loc[
    :,
    [
        "communityvisibilitystate",
        "personastate",
        "timecreated",
        "CommunityBanned",
        "NumberOfVACBans",
        "DaysSinceLastBan",
        "NumberOfGameBans",
        "EconomyBan",
        "friendsList",
        "ownedGames",
        "groups",
        "recentlyPlayed",
        "commentpermission",
        "loccountrycode",
        "avatar",
    ],
]

In [ ]:
from datetime import date, datetime, timedelta
from math import isnan

from pandas import Series, Timedelta


DEFAULT_AVATAR = (
    "https://avatars.steamstatic.com/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg"
)

data["MisdemenourCount"] = (
    data["NumberOfVACBans"]
    + data["NumberOfGameBans"]
    + data["CommunityBanned"].apply(lambda b: 1 if b else 0)
    + data["EconomyBan"].apply(lambda b: 1 if b else 0)
)


# Doesn't account for people who doesn't publicize the info
data["ActiveAfterBan"] = data["recentlyPlayed"].apply(
    lambda d: d.get("total_count", 0) > 0
) & (data["NumberOfVACBans"] > 0)

data["HasDefaultAvatar"] = data["avatar"] == DEFAULT_AVATAR
data["WeWereBadButNowWereGood"] = data["DaysSinceLastBan"] > 365

currentTimestamp = datetime.now()
data["LastBanTime"] = data["DaysSinceLastBan"].apply(
    lambda d: (currentTimestamp - timedelta(d)).timestamp() if d > 0 else None
)

data["averageTimeBetweenBans"] = (
    (data["LastBanTime"] - data["timecreated"])
    / (data["NumberOfVACBans"] + data["NumberOfGameBans"])
).apply(lambda d: None if isnan(d) else d)

data["EstimatedFirstBan"] = data["LastBanTime"] - data["averageTimeBetweenBans"] * (
    (data["NumberOfVACBans"] + data["NumberOfGameBans"]) - 1
)

data["nextBanTime"] = data["LastBanTime"] + data["averageTimeBetweenBans"]

KeyError: ''